In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install captum
!pip install fvcore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=4fb3ef3b9ab1a85dce4ecd4472a93dd34867e66872988f1a9e4adab2acd833ce
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31532 sha256=2dd1a3a54a1afe2bf84f5b3a27a8e63a154106a17da7b0b490d055d3637f3b76
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from captum.attr import visualization as viz
from fvcore.nn import FlopCountAnalysis, flop_count_table
from captum.attr import IntegratedGradients, LayerConductance, DeepLift, LayerDeepLift,LayerIntegratedGradients
import itertools
import pandas as pd
import os
import random
import numpy as np
from itertools import product
from torch.utils.data import SequentialSampler, RandomSampler
from torch.utils.data import DataLoader

In [4]:
method_names = ["LayerIntegratedGradients", "LayerDeepLift"]
INPUT_SHAPE= (1, 1, 28, 28)
KERNEL_SIZES = [3, 3, 3]
PADDING = [1, 1, 1]
Training_Device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Feature_Attribution_Device = torch.device("cpu")
NUM_EPOCHS=1
NUM_CLASSES=10
IN_CHANNELS=1
NUM_RUN=15

dataset_name = "Baseline Mnist"
folder_path = '/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage3/' + dataset_name
filename = '/Baseline Mnist 3*3_result.csv'
file_path = folder_path+filename
print(file_path)

/content/drive/My Drive/2023 InterpretingNN/code/experiment and result stage3/Baseline Mnist/Baseline Mnist 3*3_result.csv


# Model structure defination

In [5]:
class Inception(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Inception, self).__init__()
        stride = 1  # Common stride for all branches

        # Define branches with different kernel sizes and paddings
        self.branch1 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[0], stride=stride, padding=PADDING[0]),
            nn.ReLU()
        )
        self.branch2 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[1], stride=stride, padding=PADDING[1]),
            nn.ReLU()
        )
        self.branch3 = nn.Sequential(
            nn.Conv2d(in_channels, 8, kernel_size=KERNEL_SIZES[2], stride=stride, padding=PADDING[2]),
            nn.ReLU()
        )
        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=stride, padding=1),
        )


        # Fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(19600, num_classes),
        )

    def forward(self, x):
        x.requires_grad_(True)
        x1 = self.branch1(x)
        x2 = self.branch2(x)
        x3 = self.branch3(x)
        x4 = self.branch4(x)
        x = torch.cat((x1, x2, x3, x4), dim=1)  # Concatenating along the channel dimension
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = self.fc(x)
        return x


# Get dataset

In [6]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST('.', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('.', train=False, download=True, transform=transform)

100%|██████████| 9912422/9912422 [00:00<00:00, 251054994.11it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 92540636.99it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 77222485.95it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 3147287.09it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



# FLOP Count

In [7]:
def count_flops(model, input_size):
    inputs = torch.randn(input_size)
    flops = FlopCountAnalysis(model, inputs)
    return flop_count_table(flops)

inception_model = Inception(1, 10)
# Assuming the input size for CIFAR-10 (batch size, channels, height, width)
input_size = (1, 1, 28, 28)
flops_table = count_flops(inception_model, input_size)
print(flops_table)

| module             | #parameters or shape   | #flops   |
|:-------------------|:-----------------------|:---------|
| model              | 0.196M                 | 0.365M   |
|  branch1.0         |  80                    |  56.448K |
|   branch1.0.weight |   (8, 1, 3, 3)         |          |
|   branch1.0.bias   |   (8,)                 |          |
|  branch2.0         |  80                    |  56.448K |
|   branch2.0.weight |   (8, 1, 3, 3)         |          |
|   branch2.0.bias   |   (8,)                 |          |
|  branch3.0         |  80                    |  56.448K |
|   branch3.0.weight |   (8, 1, 3, 3)         |          |
|   branch3.0.bias   |   (8,)                 |          |
|  fc.0              |  0.196M                |  0.196M  |
|   fc.0.weight      |   (10, 19600)          |          |
|   fc.0.bias        |   (10,)                |          |


# Train and attribution functions

train and eval function

In [8]:
def train(epoch, model, train_loader, optimizer, criterion, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Train Loss = {train_loss / len(train_loader):.4f}, Train Accuracy = {train_accuracy:.2f}%")

    return train_accuracy

def test(epoch, model, test_loader, criterion, device):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            test_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

    test_accuracy = 100. * correct / total
    print(f"Epoch {epoch}: Test Loss = {test_loss / len(test_loader):.4f}, Test Accuracy = {test_accuracy:.2f}%")

    return test_accuracy


functions for calculate attribution

In [9]:
def print_ig(test_loader, model, device, num_batches=5):
    # Move the model to the device (CPU or CUDA)
    model.to(device)

    # Ensure the model is in evaluation mode
    model.eval()

    # Dictionary to store cumulative attributions for each layer
    cumul_ig_attributions = {}

    # Process 'num_batches' batches
    for _ in range(num_batches):
        inputs, target_class = next(iter(test_loader))
        inputs = inputs.to(device)

        # Iterate through each named module and compute attributions for Conv2d layers with learnable parameters
        for layer_name, layer_module in model.named_modules():
            # Check if the layer is a Conv2d layer with learnable parameters
            if isinstance(layer_module, nn.Conv2d) or isinstance(layer_module, nn.MaxPool2d):
                # Initialize LayerIntegratedGradients for the layer
                lig = LayerIntegratedGradients(model, layer_module)

                # Compute the attributions for the current layer
                try:
                    attributions = lig.attribute(inputs, target=target_class.to(device))
                except Exception as e:
                    print(f"Error computing attributions for layer {layer_name}: {e}")
                    continue

                # Sum up the attributions for the current layer
                attr_sum = attributions.cpu().detach().numpy().sum()

                # Add to the cumulative sum for the layer
                if layer_name in cumul_ig_attributions:
                    cumul_ig_attributions[layer_name] += attr_sum
                else:
                    cumul_ig_attributions[layer_name] = attr_sum

                # Free up memory
                del attributions, lig

    # Calculate and print the average attributions for each layer
    avg_ig_attributions = {layer: total_sum / num_batches for layer, total_sum in cumul_ig_attributions.items()}
    for layer_name, avg_attr in avg_ig_attributions.items():
        print(f'Layer: {layer_name}')
        print(f'Average Attribution: {avg_attr}')

    return avg_ig_attributions


def print_deeplift(test_loader, model, device, num_batches=5):
    # Move the model to the specified device and set it to evaluation mode
    model.to(device).eval()

    # Dictionary to store cumulative DeepLIFT attributions for each layer
    cumul_dl_attributions = {}

    # Process 'num_batches' batches
    for _ in range(num_batches):
        inputs, target_class = next(iter(test_loader))
        inputs, target_class = inputs.to(device), target_class.to(device)

        # Compute the attributions for Conv2d layers
        for layer_name, layer_module in model.named_modules():
            # Skip the whole model's container and focus on Conv2d layers with learnable parameters
            if isinstance(layer_module, nn.Conv2d) or isinstance(layer_module, nn.MaxPool2d):
                # Initialize LayerDeepLift with the current layer
                ldl = LayerDeepLift(model, layer_module)

                # Compute the attributions for the current layer
                attributions_ldl = ldl.attribute(inputs, target=target_class.to(device))

                # Sum up the attributions for the current layer
                attr_sum = attributions_ldl.cpu().data.numpy().sum()

                # Add to the cumulative sum for the layer
                if layer_name in cumul_dl_attributions:
                    cumul_dl_attributions[layer_name] += attr_sum
                else:
                    cumul_dl_attributions[layer_name] = attr_sum

                del attributions_ldl, ldl

    # Calculate and print the average attributions for each layer
    avg_dl_attributions = {layer: total_sum / num_batches for layer, total_sum in cumul_dl_attributions.items()}
    for layer_name, avg_attr in avg_dl_attributions.items():
        print(f'Layer: {layer_name}')
        print(f'Average Attribution: {avg_attr}')

    return avg_dl_attributions

# Possible Hyperparameter grid search creation

in hyperparams_list_dict, each hyperparameter has corresponding possible choices as a list, during experiment, given hyperparams sequence, location each hyperperameter's location using hyperparameter encoding function to convert strings or classes back into their index in hyperparams_list_dict

In [10]:
'''sample_hyperparams_list_dict = {
    'initial_lr': [],
    'optimizer': torch.optim.Adam,  # Example optimizer
    'criterion': torch.nn.CrossEntropyLoss(),
    'train_data_used': 0.8,
    'train_set_shuffle': True,
    'train_batch_size': 64
}'''

"sample_hyperparams_list_dict = {\n    'initial_lr': [],\n    'optimizer': torch.optim.Adam,  # Example optimizer\n    'criterion': torch.nn.CrossEntropyLoss(),\n    'train_data_used': 0.8,\n    'train_set_shuffle': True,\n    'train_batch_size': 64\n}"

In [11]:
def generate_hyperparameter_combinations(hyperparams):
    """
    Generate a sequence of hyperparameter combinations.

    :param hyperparams: A dictionary where keys are the names of hyperparameters,
                        and values are lists of possible choices for each hyperparameter.
    :return: A list of dictionaries, each representing a unique combination of hyperparameters.
    """
    # Extract the hyperparameter names and their corresponding choices
    keys, values = zip(*hyperparams.items())

    # Generate all possible combinations of hyperparameter values
    all_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

    return all_combinations

# Automated Experiments

In [12]:
def get_data_loader(hyperparams, train_dataset, test_dataset):
  shuffle = hyperparams['train_set_shuffle']
  train_batch_size = hyperparams["train_batch_size"]
  train_data_used_num = int(len(train_dataset) * hyperparams["train_data_used"])
  train_indices = torch.randperm(len(train_dataset))[:train_data_used_num]

  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=shuffle)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
  return train_loader, test_loader

In [13]:
def run_experiments(hyperparams, train_loader, test_loader, num_epochs, num_classes=NUM_CLASSES, in_channels=IN_CHANNELS):
    # Initialize model
    model = Inception(in_channels, num_classes).to(Training_Device)  # Assuming Inception is defined elsewhere

    # Use the optimizer from hyperparameters
    optimizer = hyperparams['optimizer'](model.parameters(), lr=hyperparams['initial_lr'])
    criterion = hyperparams['criterion']

    train_accuracy = {}
    test_accuracy = {}

    for epoch in range(num_epochs):
            # Assuming train and test functions are defined elsewhere
            train_acc = train(epoch, model, train_loader, optimizer, criterion, Training_Device)
            test_acc = test(epoch, model, test_loader, criterion, Training_Device)

            train_accuracy["train accuracy epoch"+str(epoch)] = train_acc
            test_accuracy["test accuracy epoch"+str(epoch)] = test_acc



    print("deeplift")
    dl_attributions = print_deeplift(test_loader, model, Feature_Attribution_Device)
    print("integrated_gradient")
    ig_attributions = print_ig(test_loader, model, Feature_Attribution_Device)
    return ig_attributions, dl_attributions, train_accuracy, test_accuracy

# Functions for saving attribution

In [14]:
def run_experiments_and_save(hyperparams_combinations, train_dataset, test_dataset, csv_file, num_run=NUM_RUN, num_epochs=NUM_EPOCHS):
    """
    Run experiments for each combination of hyperparameters, get feature layer attributions for DeepLift and Integrated Gradients,
    save the results to a CSV file after each run, and skip any combinations that have already been run.

    :param hyperparams_combinations: List of dictionaries with hyperparameter combinations.
    :param train_dataset: Training dataset.
    :param test_dataset: Test dataset.
    :param csv_file: Path to the CSV file for saving results.
    :param num_epochs: Number of epochs for training.
    """

    # Check if the CSV file exists and load existing data
    if os.path.exists(csv_file):
        existing_data = pd.read_csv(csv_file)
    else:
        existing_data = pd.DataFrame()

    for combo in hyperparams_combinations:
        print("\n\n")
        print("Combination:")
        print(combo)
        for i in range(num_run):  # For each run index
            print("runtime" + str(i))
            # Prepare data for checking if it's already processed
            combo_check = combo.copy()
            combo_check['run'] = i

            # Convert combo_check to string representation
            combo_check = {k: str(v) for k, v in combo_check.items()}



            if not existing_data.empty:
                # Filter existing_data to only the columns of interest
                filtered_data = existing_data[list(combo_check.keys())]
                # Convert filtered_data to string representation
                filtered_data = filtered_data.astype(str)
                # Check if a row with the same combination exists
                if any((filtered_data == pd.Series(combo_check)).all(axis=1)):
                    print("Combination already processed, skipping...")
                    continue  # Skip if combination is already processed

                # Set seed for reproducibility
            random.seed(i)
            np.random.seed(i)
            torch.manual_seed(i)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(i)

            train_loader, test_loader = get_data_loader(combo, train_dataset, test_dataset)

                # Run experiments and compute attributions
            ig_attributions, dl_attributions, train_accuracy, test_accuracy = run_experiments(hyperparams=combo, train_loader=train_loader, test_loader=test_loader, num_epochs=num_epochs)
            for method in ['deeplift', 'integrated_gradients']:
              if method == 'deeplift':
                  attr = dl_attributions, train_accuracy, test_accuracy
              elif method == 'integrated_gradients':
                  attr = ig_attributions, train_accuracy, test_accuracy

              combo_results = combo.copy()
              for d in attr:
                  combo_results.update(d)

              combo_results['method'] = method
              combo_results['run'] = i

              # Convert all values in combo_results to strings
              combo_results = {k: str(v) for k, v in combo_results.items()}

              # Append results to the existing data
              existing_data = existing_data.append(combo_results, ignore_index=True)

            # Save the data to CSV after each run
            existing_data.to_csv(csv_file, index=False)

# Experiments

In [15]:
hyperparams_choice_list = {
    'initial_lr': [0.001, 0.0007, 0.0003],
    'optimizer': [torch.optim.Adam], # Example optimizer
    'criterion': [torch.nn.CrossEntropyLoss()],
    'train_data_used': [1],
    'train_set_shuffle': [True],
    'train_batch_size': [64, 32, 16, 8]
}

combinations = generate_hyperparameter_combinations(hyperparams_choice_list)
for combo in combinations:
    print(combo)

{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 32}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 16}
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 8}
{'initial_lr': 0.0007, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
{'initial_lr': 0.0007, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train

In [ ]:
run_experiments_and_save(combinations, train_dataset, test_dataset, csv_file=file_path)




Combination:
{'initial_lr': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'criterion': CrossEntropyLoss(), 'train_data_used': 1, 'train_set_shuffle': True, 'train_batch_size': 64}
runtime0
Epoch 0: Train Loss = 0.2165, Train Accuracy = 93.67%
Epoch 0: Test Loss = 0.0978, Test Accuracy = 97.29%
deeplift
Layer: branch1.0
Average Attribution: 284.1470703125
Layer: branch2.0
Average Attribution: 162.65947265625
Layer: branch3.0
Average Attribution: 80.92274169921875
Layer: branch4.0
Average Attribution: 185.12291259765624
integrated_gradient
Layer: branch1.0
Average Attribution: 285.1515232258613
Layer: branch2.0
Average Attribution: 162.74044746441456
Layer: branch3.0
Average Attribution: 81.83976557609257
Layer: branch4.0
Average Attribution: 185.122918162167
runtime1


<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.2097, Train Accuracy = 93.76%
Epoch 0: Test Loss = 0.0957, Test Accuracy = 97.15%
deeplift
Layer: branch1.0
Average Attribution: 63.95699462890625
Layer: branch2.0
Average Attribution: 197.983251953125
Layer: branch3.0
Average Attribution: 141.66636962890624
Layer: branch4.0
Average Attribution: 216.5097412109375
integrated_gradient
Layer: branch1.0
Average Attribution: 64.01524850285008
Layer: branch2.0
Average Attribution: 198.51509484680903
Layer: branch3.0
Average Attribution: 141.77154407699337
Layer: branch4.0
Average Attribution: 216.50972336133472
runtime2


<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.2149, Train Accuracy = 93.77%
Epoch 0: Test Loss = 0.0939, Test Accuracy = 96.99%
deeplift
Layer: branch1.0
Average Attribution: 121.69847412109375
Layer: branch2.0
Average Attribution: 177.4898681640625
Layer: branch3.0
Average Attribution: 116.23778076171875
Layer: branch4.0
Average Attribution: 189.09588623046875
integrated_gradient
Layer: branch1.0
Average Attribution: 121.98178520452964
Layer: branch2.0
Average Attribution: 177.49931556031225
Layer: branch3.0
Average Attribution: 117.12669461003063
Layer: branch4.0
Average Attribution: 189.09588900356968
runtime3


...

<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.1882, Train Accuracy = 94.40%
Epoch 0: Test Loss = 0.1052, Test Accuracy = 96.81%
deeplift
Layer: branch1.0
Average Attribution: 126.77196044921875
Layer: branch2.0
Average Attribution: 29.56871643066406
Layer: branch3.0
Average Attribution: 198.44658203125
Layer: branch4.0
Average Attribution: 168.74224853515625
integrated_gradient
Layer: branch1.0
Average Attribution: 126.56242141139637
Layer: branch2.0
Average Attribution: 29.55119953380022
Layer: branch3.0
Average Attribution: 196.94294214624156
Layer: branch4.0
Average Attribution: 168.74223972461692
runtime14


<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)


Epoch 0: Train Loss = 0.1963, Train Accuracy = 94.10%
Epoch 0: Test Loss = 0.1023, Test Accuracy = 96.74%
deeplift
Layer: branch1.0
Average Attribution: 174.71265869140626
Layer: branch2.0
Average Attribution: 64.08074340820312
Layer: branch3.0
Average Attribution: 14.602011108398438
Layer: branch4.0
Average Attribution: 186.80914306640625
integrated_gradient
Layer: branch1.0
Average Attribution: 172.2437491022626
Layer: branch2.0
Average Attribution: 64.04126199834782
Layer: branch3.0
Average Attribution: 14.589905387993264
Layer: branch4.0
Average Attribution: 186.8091421839497


<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
<ipython-input-14-566569a8104c>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  existing_data = existing_data.append(combo_results, ignore_index=True)
